# * Anotações

In [22]:
# Anotações

# 1. Comando que retorna a frequência de dados duplicados em uma coluna
# value_counts()

# 2. Comando que retira os dados duplicados em uma coluna e retorna um df
# drop_duplicates()

# 3. Removendo coluna e salvando o no df no df original:
# df = df.drop(columns=['nome_da_coluna_que_se_deseja_remover'])

# 4. o comando .tolist() converte objetos em listas

# 0.0 Imports

In [23]:
import pandas as pd
import numpy as np
import streamlit as st
import plotly.express as px

import inflection

# 0.1. Helper Variables

In [24]:
COUNTRIES = {
    1: "India",
    14: "Australia",
    30: "Brazil",
    37: "Canada",
    94: "Indonesia",
    148: "New Zeland",
    162: "Philippines",
    166: "Qatar",
    184: "Singapure",
    189: "South Africa",
    191: "Sri Lanka",
    208: "Turkey",
    214: "United Arab Emirates",
    215: "England",
    216: "United States of America",
}


COLORS = {
    "3F7E00": "darkgreen",
    "5BA829": "green",
    "9ACD32": "lightgreen",
    "CDD614": "orange",
    "FFBA00": "red",
    "CBCBC8": "darkred",
    "FF7800": "darkred",
}

# 0.2. Helper Functions

In [25]:
def show_dataframe_dimensions(dataframe):
    print(f"Number of Rows: {dataframe.shape[0]}")
    print(f"Number of Columns: {dataframe.shape[1]}")
    
    return None

In [26]:
def get_numerical_attributes(dataframe):
    return dataframe.select_dtypes(include=['int64','float64'])

In [27]:
def get_first_order_statistics(dataframe):
    # Central Tendency Metrics
    mean = pd.DataFrame(dataframe.apply(np.mean)).T
    median = pd.DataFrame(dataframe.apply(np.median)).T

    # Dispersion Metrics
    min_ = pd.DataFrame(dataframe.apply(min)).T
    max_ = pd.DataFrame(dataframe.apply(max)).T
    range_ = pd.DataFrame(dataframe.apply(lambda x: x.max() - x.min())).T
    std = pd.DataFrame(dataframe.apply(np.std)).T
    skew = pd.DataFrame(dataframe.apply(lambda x: x.skew())).T
    kurtosis = pd.DataFrame(dataframe.apply(lambda x: x.kurtosis())).T

    # Metrics Concatenation
    m = pd.concat([min_, max_, range_, mean, median, std, skew, kurtosis]).T.reset_index()
    m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
    
    return m

In [28]:
def rename_columns(dataframe):
    df = dataframe.copy()

    title = lambda x: inflection.titleize(x)

    snakecase = lambda x: inflection.underscore(x)

    spaces = lambda x: x.replace(" ", "")

    cols_old = list(df.columns)

    cols_old = list(map(title, cols_old))

    cols_old = list(map(spaces, cols_old))

    cols_new = list(map(snakecase, cols_old))

    df.columns = cols_new

    return df

In [29]:
def country_name(country_id):
    return COUNTRIES[country_id]

In [30]:
def color_name(color_code):
    return COLORS[color_code]

In [31]:
def create_price_tye(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"

In [32]:
def adjust_columns_order(dataframe):
    df = dataframe.copy()

    new_cols_order = [
        "restaurant_id",
        "restaurant_name",
        "country",
        "city",
        "address",
        "locality",
        "locality_verbose",
        "longitude",
        "latitude",
        "cuisines",
        "price_type",
        "average_cost_for_two",
        "currency",
        "has_table_booking",
        "has_online_delivery",
        "is_delivering_now",
        "aggregate_rating",
        "rating_color",
        "color_name",
        "rating_text",
        "votes",
    ]

    return df.loc[:, new_cols_order]

In [33]:
def process_data(file_path):
    df = pd.read_csv(file_path)

    df = df.dropna()

    df = rename_columns(df)

    df["price_type"] = df.loc[:, "price_range"].apply(lambda x: create_price_tye(x))

    df["country"] = df.loc[:, "country_code"].apply(lambda x: country_name(x))

    df["color_name"] = df.loc[:, "rating_color"].apply(lambda x: color_name(x))

    df["cuisines"] = df.loc[:, "cuisines"].apply(lambda x: x.split(",")[0])

    df = df.drop_duplicates()

    df = adjust_columns_order(df)
    
    df = df.reset_index()
    
    df = df.drop(columns=['index'])

    df.to_csv("dataset/processed/data.csv", index=False)

    return df

# 0.3. Helper Configurations

In [34]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# 0.4. Load Data

In [35]:
RAW_DATA_PATH = f"dataset/raw/data.csv"
df_raw = pd.read_csv(RAW_DATA_PATH)
df_raw.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,Average Cost for two,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
1,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,"European, Asian",3100,Botswana Pula(P),0,0,0,0,4,4.7,3F7E00,Excellent,469
3,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,"Filipino, American, Italian, Bakery",800,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,867
4,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,700,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,858


# 1.0. DATA DESCRIPTION

In [36]:
df1 = df_raw.copy()
df1.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,Average Cost for two,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
1,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,"European, Asian",3100,Botswana Pula(P),0,0,0,0,4,4.7,3F7E00,Excellent,469
3,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,"Filipino, American, Italian, Bakery",800,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,867
4,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,700,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,858


# 1.1. Columns Description

|Column|Description|
|------|-----------|
|Restaurant ID|ID do restaurante|
|Restaurant Name|Nome do Restaurante|
|Country Code|Código do País|
|City|Nome da Cidade onde o restaurante está|
|Address|Endereço do restaurante|
|Locality|Localização e pontos de referência do restaurante|
|Locality Verbose|Localização e pontos de referência do restaurante (Mais informações)|
|Longitude|Ponto geográfico de Longitude do Restaurante|
|Latitude|Ponto geográfico de Latitude do Restaurante|
|Cuisines|Tipos de Culinária servidos no restaurante|
|Average Cost for two|Preço Médio de um prato para duas pessoas no restaurante|
|Currency|Moeda do país|
|Has Table booking|Se o restaurante possui serviços de reserva; 1 - Sim; 0 - Não|
|Has Online delivery|Se o restaurante possui serviços de pedido on-line; 1 - Sim; 0 - Não|
|Is delivering now|Se o restaurante faz entregas; 1 - Sim; 0 - Não|
|Switch to order menu|-|
|Price range|Variação de preços do restaurante; 1 a 4 - Quanto maior o valor, mais caro serão os pratos|
|Aggregate rating|Nota média do restaurante|
|Rating color|Código Hexadecimal da cor do restaurante com base em sua nota média|
|Rating text|Categoria em que o restaurante está com base em sua nota média|
|Votes|Quantidade de avaliações que o restaurante já recebeu|

# 1.2. Rename Columns

In [37]:
df1 = rename_columns(df1)
df1.head()

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,average_cost_for_two,currency,has_table_booking,has_online_delivery,is_delivering_now,switch_to_order_menu,price_range,aggregate_rating,rating_color,rating_text,votes
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
1,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,"European, Asian",3100,Botswana Pula(P),0,0,0,0,4,4.7,3F7E00,Excellent,469
3,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,"Filipino, American, Italian, Bakery",800,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,867
4,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,700,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,858


# 1.3. Data Dimensions

In [38]:
show_dataframe_dimensions(df1)

Number of Rows: 7527
Number of Columns: 21


# 1.4. Data Types

In [39]:
df1.dtypes

restaurant_id             int64
restaurant_name          object
country_code              int64
city                     object
address                  object
locality                 object
locality_verbose         object
longitude               float64
latitude                float64
cuisines                 object
average_cost_for_two      int64
currency                 object
has_table_booking         int64
has_online_delivery       int64
is_delivering_now         int64
switch_to_order_menu      int64
price_range               int64
aggregate_rating        float64
rating_color             object
rating_text              object
votes                     int64
dtype: object

# 1.5. Check NA

In [40]:
df1.isna().sum()

restaurant_id            0
restaurant_name          0
country_code             0
city                     0
address                  0
locality                 0
locality_verbose         0
longitude                0
latitude                 0
cuisines                15
average_cost_for_two     0
currency                 0
has_table_booking        0
has_online_delivery      0
is_delivering_now        0
switch_to_order_menu     0
price_range              0
aggregate_rating         0
rating_color             0
rating_text              0
votes                    0
dtype: int64

In [41]:
df1

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,average_cost_for_two,currency,has_table_booking,has_online_delivery,is_delivering_now,switch_to_order_menu,price_range,aggregate_rating,rating_color,rating_text,votes
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
1,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,"European, Asian",3100,Botswana Pula(P),0,0,0,0,4,4.7,3F7E00,Excellent,469
3,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,"Filipino, American, Italian, Bakery",800,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,867
4,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,700,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7522,5912546,Eataly,208,İstanbul,"Zorlu Center AVM, Köprü Katı, Levazım Mahalles...","Zorlu Center AVM, Levazım, Beşiktaş","Zorlu Center AVM, Levazım, Beşiktaş, İstanbul",29.017326,41.065322,"Italian, Pizza, Fresh Fish",300,Turkish Lira(TL),0,0,0,0,4,4.3,5BA829,Very Good,1367
7523,5913006,Tarihi Çınaraltı Aile Çay Bahçesi,208,İstanbul,"Çengelköy Mahallesi, Çınaraltı Camii Sokak, No...",Çengelköy Merkez,"Çengelköy Merkez, İstanbul",29.052620,41.050280,"Fast Food, Izgara, Seafood, Tea, Coffee",45,Turkish Lira(TL),0,0,0,0,2,4.5,3F7E00,Excellent,1172
7524,5923535,Boon Cafe & Restaurant,208,İstanbul,"Çengelköy Mahallesi, Çengelköy Caddesi, Kara S...",Çengelköy Merkez,"Çengelköy Merkez, İstanbul",29.052623,41.050717,Restaurant Cafe,140,Turkish Lira(TL),0,0,0,0,4,4.2,5BA829,Very Good,1160
7525,5914190,Kanaat Lokantası,208,İstanbul,"Sultantepe Mahallesi, Selmani Pak Caddesi, No ...",Üsküdar Merkez,"Üsküdar Merkez, İstanbul",29.016590,41.025741,"Home-made, Izgara",95,Turkish Lira(TL),0,0,0,0,3,4.0,5BA829,Very Good,770


# *All types of cuisines

In [42]:
df1.loc[:, 'cuisines'].unique().tolist()

['Italian',
 'European, Asian',
 'Filipino, American, Italian, Bakery',
 'American',
 'Filipino',
 'Korean, Grill',
 'American, Filipino, Coffee',
 'Pizza, Italian',
 'Taiwanese',
 'Japanese, Latin American',
 'Japanese',
 'Italian, Pizza',
 'American, Filipino',
 'Japanese, Sushi',
 'Coffee, American, European',
 'Japanese, Ramen',
 'American, Italian, Bakery, Coffee',
 'Chinese',
 'Japanese, Korean',
 'Coffee',
 'European, Desserts',
 'European, Steak, Bakery, Desserts',
 'Filipino, Chinese',
 'Seafood, Filipino, Asian, European',
 'European, Asian, Indian',
 'Korean',
 'Filipino, Mexican',
 'American, Burger',
 'Singaporean, Street Food',
 'Vietnamese',
 'Latin American, Street Food',
 'Filipino, Asian',
 'Korean, Korean BBQ',
 'Filipino, Coffee',
 'European, Italian',
 'American, Asian',
 'European, American',
 'European, Filipino, American',
 'American, European',
 'Coffee, Japanese',
 'Japanese, Asian',
 'Seafood, American, Mediterranean, Japanese',
 'European, Filipino',
 'Healt

# 1.6. Treat NA

In [43]:
df1 = df1.dropna()
df1.isna().sum()

restaurant_id           0
restaurant_name         0
country_code            0
city                    0
address                 0
locality                0
locality_verbose        0
longitude               0
latitude                0
cuisines                0
average_cost_for_two    0
currency                0
has_table_booking       0
has_online_delivery     0
is_delivering_now       0
switch_to_order_menu    0
price_range             0
aggregate_rating        0
rating_color            0
rating_text             0
votes                   0
dtype: int64

# 1.7. Change Types

Não há a necessidade de fazer a troca de tipos dos dados.

# 1.8. Descriptive Statistics

In [44]:
numerical_data = get_numerical_attributes(df1)

# 1.8.1. Numerical Data

In [45]:
get_first_order_statistics(numerical_data)

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,restaurant_id,549.000000,1.904028e+07,1.903973e+07,1.054327e+07,7.701329e+06,7.075017e+06,-0.084078,-1.726900
1,country_code,1.000000,2.160000e+02,2.150000e+02,9.313472e+01,3.000000e+01,9.902019e+01,0.257296,-1.849704
2,longitude,-122.700455,1.753106e+02,2.980110e+02,3.376321e+01,7.378280e+01,7.736959e+01,-0.679659,-0.596239
3,latitude,-41.330428,5.597698e+01,9.730741e+01,1.931223e+01,2.523716e+01,2.327263e+01,-1.080186,0.608968
4,average_cost_for_two,0.000000,2.500002e+07,2.500002e+07,7.165669e+03,2.900000e+02,2.908773e+05,84.494674,7257.443719
5,has_table_booking,0.000000,1.000000e+00,1.000000e+00,6.230032e-02,0.000000e+00,2.417002e-01,3.622563,11.125924
6,has_online_delivery,0.000000,1.000000e+00,1.000000e+00,3.531683e-01,0.000000e+00,4.779544e-01,0.614540,-1.622773
7,is_delivering_now,0.000000,1.000000e+00,1.000000e+00,1.746539e-01,0.000000e+00,3.796708e-01,1.714175,0.938646
8,switch_to_order_menu,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000
9,price_range,1.000000,4.000000e+00,3.000000e+00,2.625399e+00,3.000000e+00,9.595017e-01,-0.041458,-0.982738


# 2. ANSWERS

In [46]:
df2 = process_data(RAW_DATA_PATH)
df2

,restaurant_id,restaurant_name,country,city,address,locality,locality_verbose,longitude,latitude,cuisines,price_type,average_cost_for_two,currency,has_table_booking,has_online_delivery,is_delivering_now,aggregate_rating,rating_color,color_name,rating_text,votes
0,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,expensive,1100,Botswana Pula(P),1,0,0,4.6,3F7E00,darkgreen,Excellent,619
1,6314542,Blackbird,Philippines,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,gourmet,3100,Botswana Pula(P),0,0,0,4.7,3F7E00,darkgreen,Excellent,469
2,6301293,Banapple,Philippines,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,Filipino,expensive,800,Botswana Pula(P),0,0,0,4.4,5BA829,green,Very Good,867
3,6315689,Bad Bird,Philippines,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,expensive,700,Botswana Pula(P),0,0,0,4.4,5BA829,green,Very Good,858
4,6304833,Manam,Philippines,Makati City,"Level 1, Greenbelt 2, Ayala Center, Greenbelt,...","Greenbelt 2, San Lorenzo, Makati City","Greenbelt 2, San Lorenzo, Makati City, Makati ...",121.020380,14.552351,Filipino,expensive,700,Botswana Pula(P),0,0,0,4.7,3F7E00,darkgreen,Excellent,930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6924,5912546,Eataly,Turkey,İstanbul,"Zorlu Center AVM, Köprü Katı, Levazım Mahalles...","Zorlu Center AVM, Levazım, Beşiktaş","Zorlu Center AVM, Levazım, Beşiktaş, İstanbul",29.017326,41.065322,Italian,gourmet,300,Turkish Lira(TL),0,0,0,4.3,5BA829,green,Very Good,1367
6925,5913006,Tarihi Çınaraltı Aile Çay Bahçesi,Turkey,İstanbul,"Çengelköy Mahallesi, Çınaraltı Camii Sokak, No...",Çengelköy Merkez,"Çengelköy Merkez, İstanbul",29.052620,41.050280,Fast Food,normal,45,Turkish Lira(TL),0,0,0,4.5,3F7E00,darkgreen,Excellent,1172
6926,5923535,Boon Cafe & Restaurant,Turkey,İstanbul,"Çengelköy Mahallesi, Çengelköy Caddesi, Kara S...",Çengelköy Merkez,"Çengelköy Merkez, İstanbul",29.052623,41.050717,Restaurant Cafe,gourmet,140,Turkish Lira(TL),0,0,0,4.2,5BA829,green,Very Good,1160
6927,5914190,Kanaat Lokantası,Turkey,İstanbul,"Sultantepe Mahallesi, Selmani Pak Caddesi, No ...",Üsküdar Merkez,"Üsküdar Merkez, İstanbul",29.016590,41.025741,Home-made,expensive,95,Turkish Lira(TL),0,0,0,4.0,5BA829,green,Very Good,770


# Geral

## 1. Quantos restaurantes únicos estão registrados?

In [47]:
df2_copy = df2.copy()
df2_copy.loc[:,'cuisines'].unique().tolist()

['Italian',
 'European',
 'Filipino',
 'American',
 'Korean',
 'Pizza',
 'Taiwanese',
 'Japanese',
 'Coffee',
 'Chinese',
 'Seafood',
 'Singaporean',
 'Vietnamese',
 'Latin American',
 'Healthy Food',
 'Cafe',
 'Fast Food',
 'Brazilian',
 'Argentine',
 'Arabian',
 'Bakery',
 'Tex-Mex',
 'Bar Food',
 'International',
 'French',
 'Steak',
 'German',
 'Sushi',
 'Grill',
 'Peruvian',
 'North Eastern',
 'Ice Cream',
 'Burger',
 'Mexican',
 'Vegetarian',
 'Contemporary',
 'Desserts',
 'Juices',
 'Beverages',
 'Spanish',
 'Thai',
 'Indian',
 'Mineira',
 'BBQ',
 'Mongolian',
 'Portuguese',
 'Greek',
 'Asian',
 'Author',
 'Gourmet Fast Food',
 'Lebanese',
 'Modern Australian',
 'African',
 'Coffee and Tea',
 'Australian',
 'Middle Eastern',
 'Malaysian',
 'Tapas',
 'New American',
 'Pub Food',
 'Southern',
 'Diner',
 'Donuts',
 'Southwestern',
 'Sandwich',
 'Irish',
 'Mediterranean',
 'Cafe Food',
 'Korean BBQ',
 'Fusion',
 'Canadian',
 'Breakfast',
 'Cajun',
 'New Mexican',
 'Belgian',
 'Cuban

In [48]:
# 1. Quantos restaurantes únicos estão registrados?
df2['restaurant_id'].nunique()

6929

## 2. Quantos países únicos estão registrados?

In [49]:
# 2. Quantos países únicos estão registrados?
df2["country"].nunique()

15

# 3. Quantas cidades únicas estão registradas?

In [50]:
# 3. Quantas cidades únicas estão registradas?
df2["city"].nunique()

# ou df2.loc[:,"city"].nunique()

125

# 4. Qual o total de avaliações feitas?

In [51]:
# 4. Qual o total de avaliações feitas?
df2["votes"].sum()

4194533

# 5. Qual o total de tipos de culinária registrados?

In [52]:
df2.columns

Index(['restaurant_id', 'restaurant_name', 'country', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'price_type', 'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'aggregate_rating',
       'rating_color', 'color_name', 'rating_text', 'votes'],
      dtype='object')

In [53]:
# 5. Qual o total de tipos de culinária registrados?
df2["cuisines"].nunique()

165

# País

## 1. Qual o nome do país que possui mais cidades registradas?

In [54]:
# 1. Qual o nome do país que possui mais cidades registradas?

# sape
# sa...ída
## o nome do país que possui a maior quantidade de cidades registradas na base de dados

# p...processo
# 'selecionar' a coluna cidades e 'agrupar por' país, dentro desse agrupamento 'localizar'
# o país que possui o maior número de cidades agrupadas

# e...ntrada
# v1

(df2.loc[:,["city","country"]]
    .groupby("country")
    .nunique()
    .sort_values("city",ascending=False)
    .reset_index()
    .iloc[0,0])

'India'

## 2. Qual o nome do país que possui mais restaurantes registrados?

In [55]:
(df2.loc[:,["country","restaurant_id"]]
    .groupby("country")
    .nunique()
    .sort_values("restaurant_id",ascending=False)
    .reset_index()
    .head(10))
    #.iloc[0,0])

,country,restaurant_id
0,India,3111
1,United States of America,1374
2,England,400
3,South Africa,346
4,United Arab Emirates,300
5,Brazil,240
6,New Zeland,239
7,Australia,180
8,Canada,180
9,Turkey,159


## 3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 registrados?

In [56]:
(df2.loc[df2["price_type"]=="gourmet",["country","price_type"]]
    .groupby("country")
    .nunique()
    .sort_values("price_type",ascending=False)
    .reset_index()
    .iloc[0,0])

'Australia'

## 4. Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?

In [57]:
(df2.loc[:,["cuisines","country"]]
    .groupby("country")
    .nunique()
    .sort_values("cuisines",ascending=False)
    .reset_index()
    .iloc[0,0])        

'India'

## 5. Qual o nome do país que possui a maior quantidade de avaliações feitas?

In [58]:
(df2.loc[:,["votes","country"]]
    .groupby("country")
    .sum()
    .sort_values("votes", ascending=False)
    .reset_index()
    .iloc[0,0])

'India'

## 6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?

In [59]:
(df2.loc[df2['is_delivering_now']==1,['restaurant_id','country']]
    .groupby('country')
    .count()
    .sort_values('restaurant_id',ascending=False)
    .reset_index()
    .iloc[0,0])

'India'

## 7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?

In [60]:
(df2.loc[df2['has_table_booking']==1,['restaurant_id','country']]
    .groupby('country')
    .count()
    .sort_values('restaurant_id',ascending=False)
    .reset_index()
    .iloc[0,0])

'India'

## 8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?

In [61]:
(df2.loc[:,['votes','country']]
    .groupby('country')
    .mean()
    .sort_values('votes',ascending=False)
    .reset_index())
    #.iloc[0,0])

,country,votes
0,Indonesia,1112.825000
1,India,900.084860
2,Australia,724.205556
3,Turkey,630.144654
4,Philippines,604.975000
5,United Arab Emirates,591.880000
6,Canada,583.433333
7,United States of America,380.165939
8,Qatar,376.325000
9,South Africa,236.817919


## 9. Qual o nome do país que possui, na média, a maior nota média registrada?

In [62]:
(df2.loc[:,['aggregate_rating','country']]
    .groupby('country')
    .mean()
    .sort_values('aggregate_rating',ascending=False)
    .reset_index()
    .iloc[0,0])

'Indonesia'

## 10. Qual o nome do país que possui, na média, a menor nota média registrada?

In [63]:
(df2.loc[:,['aggregate_rating','country']]
    .groupby('country')
    .mean()
    .sort_values('aggregate_rating',ascending=True)
    .reset_index()
    .iloc[0,0])

'Brazil'

## 11. Qual a média de preço de um prato para dois por país?

In [64]:
# unidade monetáia varia por país, optei por deixar como está

(df2.loc[:,['average_cost_for_two','country']]
    .groupby('country')
    .mean()
    .sort_values('average_cost_for_two', ascending=False)
    .reset_index())

,country,average_cost_for_two
0,Indonesia,303000.000000
1,Australia,138959.783333
2,Sri Lanka,2579.375000
3,Philippines,1227.825000
4,India,704.400514
5,South Africa,339.228324
6,Qatar,174.000000
7,United Arab Emirates,153.716667
8,Singapure,141.437500
9,Brazil,138.812500


# Cidade

## 1. Qual o nome da cidade que possui mais restaurantes registrados?

In [65]:

    (df2.loc[:,['restaurant_id','city','country']]
    .groupby(['city','country'])
    .count()
    .sort_values('restaurant_id',ascending=False)
    .reset_index()
    .head(10))

,city,country,restaurant_id
0,Abu Dhabi,United Arab Emirates,80
1,Mysore,India,80
2,Glasgow,England,80
3,Goa,India,80
4,Guwahati,India,80
5,Hamilton,New Zeland,80
6,Houston,United States of America,80
7,Indore,India,80
8,Jaipur,India,80
9,Kanpur,India,80


## 2. Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?

In [66]:
# no gab considerou notas >=4. Optei por deixar >4 por estar mais coerente com a pergunta de negócio.

# (df2.loc[df2['aggregate_rating']>4,['restaurant_id','city']]
#     .groupby('city')
#     .count()
#     .sort_values('restaurant_id',ascending=False)
#     .reset_index()
#     .iloc[0,0])

(df2.loc[df2['aggregate_rating']>4,['restaurant_id','city','country']]
    .groupby(['city','country'])
    .count()
    .sort_values('restaurant_id',ascending=False)
    .reset_index()
    .head(7))

,city,country,restaurant_id
0,Bangalore,India,79
1,London,England,78
2,Houston,United States of America,75
3,Jakarta,Indonesia,74
4,Chennai,India,72
5,Auckland,New Zeland,72
6,Kolkata,India,70


### *restaurantes brasileiros no banco de dados, agrupador por cidade e avaliação (fiz por conta)

In [67]:
df2.loc[df2['country']=='Brazil',['aggregate_rating','city','restaurant_name']].sort_values('aggregate_rating',ascending=False)

,aggregate_rating,city,restaurant_name
249,4.9,São Paulo,Skye - Hotel Unique
191,4.9,Rio de Janeiro,Braseiro da Gávea
273,4.9,São Paulo,Z Deli Sandwich Shop
230,4.9,Rio de Janeiro,Aprazível
177,4.9,Rio de Janeiro,Churrascaria Palace
218,4.9,Rio de Janeiro,Sushi Leblon
271,4.8,São Paulo,D.O.M.
317,4.8,São Paulo,Mocotó
197,4.8,Rio de Janeiro,Garota de Ipanema
166,4.8,Rio de Janeiro,Irajá Gastrô


## 3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?

In [68]:
# (df2.loc[df2['aggregate_rating']<2.5,['restaurant_id','city']]
#     .groupby('city')
#     .count()
#     .sort_values('restaurant_id',ascending=False)
#     .reset_index()
#     .iloc[0,0])

(df2.loc[df2['aggregate_rating']<2.5,['restaurant_id','city','country']]
    .groupby(['city','country'])
    .count()
    .sort_values('restaurant_id',ascending=False)
    .reset_index()
    .head(7))

,city,country,restaurant_id
0,Gangtok,India,33
1,Ooty,India,19
2,São Paulo,Brazil,16
3,Brasília,Brazil,15
4,Rio de Janeiro,Brazil,12
5,Manchester,England,7
6,Clarens,South Africa,4


## 4. Qual o nome da cidade que possui o maior valor médio de um prato para dois?

In [69]:
# unidade monetáia varia por país, optei por deixar como está

(df2.loc[:,['average_cost_for_two','city']]
    .groupby('city')
    .mean()
    .sort_values('average_cost_for_two',ascending=False)
    .reset_index()
    .iloc[0,0])

'Adelaide'

## 5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?

In [70]:
# (df2.loc[:,['cuisines','city']]
#     .groupby('city')
#     .nunique()
#     .sort_values('cuisines',ascending=False)
#     .reset_index()
#     .iloc[0,0])

(df2.loc[:,['cuisines','city','country']]
    .groupby(['city','country'])
    .nunique()
    .sort_values('cuisines',ascending=False)
    .reset_index()
    .head(10))

,city,country,cuisines
0,Birmingham,England,32
1,Doha,Qatar,31
2,Montreal,Canada,30
3,São Paulo,Brazil,30
4,Manchester,England,30
5,Houston,United States of America,30
6,Perth,Australia,29
7,Philadelphia,United States of America,29
8,Portland,United States of America,28
9,Calgary,Canada,28


## 6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?

In [71]:
df2.columns

Index(['restaurant_id', 'restaurant_name', 'country', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'price_type', 'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'aggregate_rating',
       'rating_color', 'color_name', 'rating_text', 'votes'],
      dtype='object')

In [72]:
(df2.loc[df2['has_table_booking']==1,['restaurant_id','city']]
    .groupby('city')
    .count()
    .sort_values('restaurant_id',ascending=False)
    .reset_index()
    .iloc[0,0])

'Bangalore'

## 7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?

In [73]:
(df2.loc[df2['is_delivering_now']==1,['restaurant_id','city']]
    .groupby('city')
    .count()
    .sort_values('restaurant_id',ascending=False)
    .reset_index()
    .iloc[0,0])

'Vadodara'

## 8. Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?

In [74]:
(df2.loc[df2['has_online_delivery']==1,['restaurant_id','city']]
    .groupby('city')
    .count()    
    .sort_values('restaurant_id',ascending=False)
    .reset_index()
    .iloc[0,0])

'Bhopal'

# Restaurantes

## 1. Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [75]:
(df2.loc[:,['restaurant_name','votes']]
    .sort_values('votes',ascending=False)
    .iloc[0,0])

'Bawarchi'

## 2. Qual o nome do restaurante com a maior nota média?

In [76]:
# se houver empate, pega o restaurante com menor id (mais antigo registro na base)

(df2.loc[:,['restaurant_name','aggregate_rating','restaurant_id']]
    .sort_values(['aggregate_rating','restaurant_id'],ascending=[False,True])
    .iloc[0,0])

'Indian Grill Room'

## 3. Qual o nome do restaurante que possui o maior valor de uma prato para duas pessoas?

In [77]:
df2['restaurant_id'].count()

6929

In [78]:
df2.columns

Index(['restaurant_id', 'restaurant_name', 'country', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'price_type', 'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'aggregate_rating',
       'rating_color', 'color_name', 'rating_text', 'votes'],
      dtype='object')

In [79]:
(df2.loc[:,['average_cost_for_two','restaurant_name']]
    .sort_values('average_cost_for_two', ascending=False)
    .iloc[0,1])

"d'Arry's Verandah Restaurant"

## 4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?

In [80]:
(df2.loc[df2['cuisines']=='Brazilian',['restaurant_name','aggregate_rating','restaurant_id']]
    .sort_values(['aggregate_rating','restaurant_id'],ascending=[True,True])
    .iloc[0,0])

'Loca Como tu Madre'

## 5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?

In [81]:
(df2.loc[df2['cuisines']=='Brazilian',['restaurant_name','aggregate_rating','restaurant_id']]
    .sort_values(['aggregate_rating','restaurant_id'],ascending=[False,True])
    .iloc[0,0])

'Braseiro da Gávea'

## 6. Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?

In [82]:
print(df2.loc[df2['has_online_delivery']==0,['restaurant_id','votes']]['votes'].mean())
print(df2.loc[df2['has_online_delivery']==1,['restaurant_id','votes']]['votes'].mean())
print('sim')

479.4210175516552
838.8216639209226
sim


In [83]:
# outra fomra

#df2.loc[:,['votes','has_online_delivery']].groupby('has_online_delivery').mean().reset_index()

## 7. Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?

In [84]:
#desconsiderando unidade monetária que varia por país...

(df2.loc[:,['average_cost_for_two','has_table_booking']]
    .groupby('has_table_booking')
    .mean()
    .sort_values('average_cost_for_two',ascending=False)
    .reset_index())

,has_table_booking,average_cost_for_two
0,1,69998.423810
1,0,3488.596866


## 8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?

In [85]:
print('Valor médio de prato para duas pessoas dos restaurantes do tipo de culinária japonesa dos Estados Unidos da América')
print(df2.loc[(df2['cuisines']=='Japanese')&(df2['country']=='United States of America'),'average_cost_for_two'].mean())
print('Valor médio de prato para duas pessoas das churrascarias americanas (BBQ)')
print(df2.loc[(df2['cuisines']=='BBQ')&(df2['country']=='United States of America'),'average_cost_for_two'].mean())

Valor médio de prato para duas pessoas dos restaurantes do tipo de culinária japonesa dos Estados Unidos da América
56.40625
Valor médio de prato para duas pessoas das churrascarias americanas (BBQ)
39.642857142857146


In [86]:
# outra maneira de fazer

# cond = (df2['country'] == 'United States of America') & ((df2['cuisines']=='BBQ') | (df2['cuisines']=='Japanese'))

# (df2.loc[cond,['average_cost_for_two','cuisines']]
#     .groupby('cuisines')
#     .mean()
#     .sort_values('average_cost_for_two', ascending=False)
#     .reset_index())

# Tipos de Culinária

## 1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?

In [87]:
# (df2.loc[df2['cuisines']=='Italian',['restaurant_id','aggregate_rating','restaurant_name']]
#     .groupby('restaurant_name')
#     .mean()
#     .sort_values(['aggregate_rating','restaurant_id'],ascending=[False,True])
#     .reset_index()
#     .iloc[0,0])

# (df2.loc[df2['cuisines']=='Italian',['restaurant_id','aggregate_rating','restaurant_name']]
#                         .groupby('restaurant_name')
#                         .mean()
#                         .sort_values(['aggregate_rating','restaurant_id'],ascending=[False,True])
#                         .reset_index()
#                         .iloc[0,2])

df_aux = (df2.loc[df2['cuisines']=='Italian',['restaurant_id','aggregate_rating','restaurant_name']]
                        .groupby('restaurant_name')
                        .mean()
                        .sort_values(['aggregate_rating','restaurant_id'],ascending=[False,True])
                        .reset_index())

df_aux
# nome_rest = df_aux.iloc[0,0]
# nota_rest = df_aux.iloc[0,2]

# print(f'Italiana: {nome_rest}', f'\n {nota_rest}/5.0')

,restaurant_name,restaurant_id,aggregate_rating
0,Cafe Del Sol Classico,6.501298e+06,4.900000
1,Ombra,7.100171e+06,4.900000
2,Celino's,7.700796e+06,4.900000
3,Andre's Cucina & Polenta Bar,1.658768e+07,4.900000
4,Di Rienzo Grocery & Deli,1.666342e+07,4.900000
5,Chicago Pizza & Oven Grinder Company,1.673318e+07,4.900000
6,Regina Pizzeria,1.679756e+07,4.900000
7,Bottega Louie,1.683168e+07,4.900000
8,Perricone's Marketplace & Café,1.692346e+07,4.900000
9,Big Bill's,1.696720e+07,4.900000


## 2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?

In [88]:
(df2.loc[df2['cuisines']=='Italian',['restaurant_id','restaurant_name','aggregate_rating']]
    .groupby('restaurant_name')
    .mean()
    .sort_values(['aggregate_rating','restaurant_id'],ascending=[True,True])
    .reset_index()
    .iloc[0,0])

'Avenida Paulista'

## 3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?

In [89]:
cond_line = df2['cuisines'] == 'American'
cols = ['restaurant_id','restaurant_name','aggregate_rating']


(df2.loc[cond_line,cols]
    .groupby('restaurant_name')
    .mean()
    .sort_values(['aggregate_rating','restaurant_id'],ascending=[False,True])
    .reset_index()
    .iloc[0,0])

'OEB Breakfast Co.'

## 4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?

In [90]:
(df2.loc[df2['cuisines']=='American',['restaurant_id','restaurant_name','aggregate_rating']]
    .groupby('restaurant_name')
    .mean()
    .sort_values(['aggregate_rating','restaurant_id'],ascending=[True,True])
    .reset_index()
    .iloc[0,0])

'Alston Bar & Beef'

## 5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?

In [104]:
df_aux = (df2.loc[df2['cuisines'] == 'Arabian',['restaurant_id','restaurant_name','aggregate_rating']]
                     .groupby('restaurant_name')
                     .mean()
                     .sort_values(['aggregate_rating','restaurant_id'],ascending=[False,True])
                     .reset_index())
        
nome_rest = df_aux.iloc[0,0]
nota_rest = df_aux.iloc[0,2]

print(nome_rest)

Mandi@36


In [91]:
(df2.loc[df2['cuisines']=='Arabian',['restaurant_id','restaurant_name','aggregate_rating']]
    .groupby('restaurant_name')
    .mean()
    .sort_values(['aggregate_rating','restaurant_id'], ascending=[False,True])
    .reset_index()
    .iloc[0,0])

'Mandi@36'

## 6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?

In [105]:
print(df2.loc[:,'country'].unique().tolist())

['Philippines', 'Brazil', 'Australia', 'United States of America', 'Canada', 'Singapure', 'United Arab Emirates', 'India', 'Indonesia', 'New Zeland', 'England', 'Qatar', 'South Africa', 'Sri Lanka', 'Turkey']


In [92]:
(df2.loc[df2['cuisines']=='Arabian',['restaurant_id','restaurant_name','aggregate_rating']]
    .groupby('restaurant_name')
    .mean()
    .sort_values(['aggregate_rating','restaurant_id'],ascending=[True,True])
    .reset_index()
    .iloc[0,0])

'Raful'

## 7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a maior média de avaliação?

In [93]:
(df2.loc[df2['cuisines']=='Japanese',['restaurant_id','restaurant_name','aggregate_rating']]
    .groupby('restaurant_name')
    .mean()
    .sort_values(['aggregate_rating','restaurant_id'],ascending=[False,True])
    .reset_index()
    .iloc[0,0])

'Sushi Samba'

## 8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a menor média de avaliação?

In [94]:
(df2.loc[df2['cuisines']=='Japanese',['restaurant_id','restaurant_name','aggregate_rating']].groupby('restaurant_name')
    .mean()
    .sort_values(['aggregate_rating','restaurant_id'],ascending=[True,True])
    .reset_index()
    .iloc[0,0])

'Banzai Sushi'

## 9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a maior média de avaliação?

In [95]:
(df2.loc[df2['cuisines']=='Home-made',['restaurant_id','restaurant_name','aggregate_rating']]
    .sort_values('aggregate_rating',ascending=False)
    .reset_index()
    .iloc[0,2])

'Kanaat Lokantası'

## 10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a menor média de avaliação?

In [96]:
(df2.loc[df2['cuisines']=='Home-made',['restaurant_name','aggregate_rating']]
    .sort_values('aggregate_rating',ascending=True)
    .iloc[0,0])

'GurMekan Restaurant'

## 11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?

In [97]:
df2.columns

Index(['restaurant_id', 'restaurant_name', 'country', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'price_type', 'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'aggregate_rating',
       'rating_color', 'color_name', 'rating_text', 'votes'],
      dtype='object')

In [98]:
# maior valor médio
(df2.loc[:,['cuisines','average_cost_for_two']]
    .groupby('cuisines')
    .mean()
    .sort_values('average_cost_for_two',ascending=False)
    .reset_index()
    .head(1))

# maior valor
#df2.loc[:,['cuisines','average_cost_for_two']].groupby('cuisines').max().sort_values('average_cost_for_two',ascending=False).reset_index()

,cuisines,average_cost_for_two
0,Modern Australian,1.470693e+06


## 12. Qual o tipo de culinária que possui a maior nota média?

In [99]:
(df2.loc[:,['cuisines','aggregate_rating']]
    .groupby('cuisines')
    .mean()
    .sort_values('aggregate_rating',ascending=False)
    .reset_index()
    .iloc[0,0])

'Others'

## 13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas?

In [100]:
line_cond = (df2['has_online_delivery']==1) & (df2['is_delivering_now']==1)
cols = ['cuisines','restaurant_id']

df2.loc[line_cond,cols].groupby('cuisines').count().sort_values('restaurant_id',ascending=False).reset_index().head(1)

,cuisines,restaurant_id
0,North Indian,317
